# Machine Learning Zoomcamp - Homework 2: Linear Regression

This notebook contains solutions to the regression homework questions from the ML Zoomcamp course.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

In [2]:
# Load the dataset
print("Loading dataset...")
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'
df = pd.read_csv(url)

# Select only the required columns
columns_to_keep = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[columns_to_keep]

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())
print("\nDataset info:")
print(df.info())

Loading dataset...
Dataset loaded successfully!
Shape: (9704, 5)

First few rows:
   engine_displacement  horsepower  vehicle_weight  model_year  \
0                  170       159.0     3413.433759        2003   
1                  130        97.0     3149.664934        2007   
2                  170        78.0     3079.038997        2018   
3                  220         NaN     2542.392402        2009   
4                  210       140.0     3460.870990        2009   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3            16.912736  
4            12.488369  

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   horsepower           8996 non-null   float64
 2   vehicle_weight       9704 non-null   float64


## Exploratory Data Analysis (EDA)

Let's first explore the dataset and understand the distribution of our target variable.

In [3]:
# EDA: Check if fuel_efficiency_mpg has a long tail
print("=== Target Variable Analysis: fuel_efficiency_mpg ===")
print(df['fuel_efficiency_mpg'].describe())

# Calculate skewness
skewness = stats.skew(df['fuel_efficiency_mpg'])
print(f"\nSkewness: {skewness:.3f}")

# Analyze distribution
median = df['fuel_efficiency_mpg'].median()
mean = df['fuel_efficiency_mpg'].mean()
print(f"Mean: {mean:.3f}")
print(f"Median: {median:.3f}")

if abs(skewness) < 0.5:
    print("\n✗ NO: The distribution does NOT have a significant long tail (approximately symmetric)")
elif skewness > 0.5:
    print("\n✓ YES: The distribution has a LONG RIGHT TAIL")
else:
    print("\n✓ YES: The distribution has a LONG LEFT TAIL")

=== Target Variable Analysis: fuel_efficiency_mpg ===
count    9704.000000
mean       14.985243
std         2.556468
min         6.200971
25%        13.267459
50%        15.006037
75%        16.707965
max        25.967222
Name: fuel_efficiency_mpg, dtype: float64

Skewness: -0.012
Mean: 14.985
Median: 15.006

✗ NO: The distribution does NOT have a significant long tail (approximately symmetric)


## Question 1: Missing Values

Which column has missing values?

In [4]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:")
for col in df.columns:
    missing_count = missing_values[col]
    print(f"{col}: {missing_count}")

# Find which column has missing values
columns_with_missing = missing_values[missing_values > 0]
if len(columns_with_missing) > 0:
    print(f"\nColumn(s) with missing values: {list(columns_with_missing.index)}")
else:
    print("\nNo missing values found in any column")

Missing values per column:
engine_displacement: 0
horsepower: 708
vehicle_weight: 0
model_year: 0
fuel_efficiency_mpg: 0

Column(s) with missing values: ['horsepower']


## Question 2: Median of Horsepower

What's the median value of the 'horsepower' column?

In [5]:
# Find median of horsepower
horsepower_median = df['horsepower'].median()
print(f"Median of horsepower: {horsepower_median}")

Median of horsepower: 149.0


## Question 3: Missing Value Strategy Comparison

Compare the RMSE when filling missing values with 0 vs filling with the mean value.

In [6]:
# Calculate mean of horsepower (excluding missing values)
horsepower_mean = df['horsepower'].mean()
print(f"Mean of horsepower: {horsepower_mean}")

# Create two versions of the dataset
# Version 1: Fill missing values with 0
df_filled_0 = df.copy()
df_filled_0['horsepower'] = df_filled_0['horsepower'].fillna(0)

# Version 2: Fill missing values with mean
df_filled_mean = df.copy()
df_filled_mean['horsepower'] = df_filled_mean['horsepower'].fillna(horsepower_mean)

# Prepare features and target for both versions
def prepare_data(df_input):
    X = df_input.drop('fuel_efficiency_mpg', axis=1)
    y = df_input['fuel_efficiency_mpg']
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Split data for both versions
X_train_0, X_test_0, y_train_0, y_test_0 = prepare_data(df_filled_0)
X_train_mean, X_test_mean, y_train_mean, y_test_mean = prepare_data(df_filled_mean)

# Train linear regression models
lr_0 = LinearRegression()
lr_0.fit(X_train_0, y_train_0)
y_pred_0 = lr_0.predict(X_test_0)
rmse_0 = np.sqrt(mean_squared_error(y_test_0, y_pred_0))

lr_mean = LinearRegression()
lr_mean.fit(X_train_mean, y_train_mean)
y_pred_mean = lr_mean.predict(X_test_mean)
rmse_mean = np.sqrt(mean_squared_error(y_test_mean, y_pred_mean))

print(f"RMSE with missing values filled with 0: {rmse_0:.3f}")
print(f"RMSE with missing values filled with mean: {rmse_mean:.3f}")
print(f"Difference: {abs(rmse_0 - rmse_mean):.3f}")

Mean of horsepower: 149.65729212983547
RMSE with missing values filled with 0: 0.530
RMSE with missing values filled with mean: 0.470
Difference: 0.060


## Question 4: Best Regularization Parameter

Find the best regularization parameter (alpha) for Ridge regression using validation dataset.

In [7]:
# Question 4: Start fresh - Fill missing values with 0 for this question
print("=== Question 4: Ridge Regression with different alpha values ===")

# Step 1: Fill missing values with 0
df_q4 = df.copy()
df_q4['horsepower'] = df_q4['horsepower'].fillna(0)

print(f"After filling NAs with 0:")
print(f"Missing values in horsepower: {df_q4['horsepower'].isnull().sum()}")
print(f"Dataset shape: {df_q4.shape}")

# Step 2: Prepare features and target
X_q4 = df_q4.drop('fuel_efficiency_mpg', axis=1)
y_q4 = df_q4['fuel_efficiency_mpg']

print(f"\nFeatures shape: {X_q4.shape}")
print(f"Target shape: {y_q4.shape}")
print(f"\nFeature columns: {list(X_q4.columns)}")

=== Question 4: Ridge Regression with different alpha values ===
After filling NAs with 0:
Missing values in horsepower: 0
Dataset shape: (9704, 5)

Features shape: (9704, 4)
Target shape: (9704,)

Feature columns: ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']


In [8]:
# Step 3: Split the data into train and validation sets
# Note: We use validation set to evaluate different alpha values
X_train_q4, X_val_q4, y_train_q4, y_val_q4 = train_test_split(
    X_q4, y_q4, test_size=0.2, random_state=42
)

print(f"Training set - Features: {X_train_q4.shape}, Target: {y_train_q4.shape}")
print(f"Validation set - Features: {X_val_q4.shape}, Target: {y_val_q4.shape}")

# Verify no missing values in training and validation data
print(f"\\nMissing values in training features: {X_train_q4.isnull().sum().sum()}")
print(f"Missing values in validation features: {X_val_q4.isnull().sum().sum()}")
print(f"Missing values in training target: {y_train_q4.isnull().sum()}")
print(f"Missing values in validation target: {y_val_q4.isnull().sum()}")

Training set - Features: (7763, 4), Target: (7763,)
Validation set - Features: (1941, 4), Target: (1941,)
\nMissing values in training features: 0
Missing values in validation features: 0
Missing values in training target: 0
Missing values in validation target: 0


In [9]:
# Step 4: Test different regularization parameters (r values)
# Following the exact instructions: [0, 0.01, 0.1, 1, 5, 10, 100]
# Evaluate models on VALIDATION dataset and round RMSE to 2 decimal places

r_values = [0, 0.01, 0.1, 1, 5, 10, 100]
rmse_scores = []

print("Testing different r values for regularized linear regression:")
print("=" * 60)
print(f"{'Model':<20} | {'r (alpha)':<10} | {'RMSE (Validation)':<15}")
print("=" * 60)

for r in r_values:
    if r == 0:
        # Linear Regression (no regularization)
        model = LinearRegression()
        model_name = "Linear Regression"
    else:
        # Ridge Regression
        model = Ridge(alpha=r)
        model_name = f"Ridge Regression"
    
    # Train the model on training set
    model.fit(X_train_q4, y_train_q4)
    
    # Make predictions on VALIDATION set
    y_pred_val = model.predict(X_val_q4)
    
    # Calculate RMSE on validation set and round to 2 decimal places
    rmse_val = np.sqrt(mean_squared_error(y_val_q4, y_pred_val))
    rmse_rounded = round(rmse_val, 2)
    rmse_scores.append(rmse_rounded)
    
    print(f"{model_name:<20} | {r:<10} | {rmse_rounded:<15}")

print("=" * 60)

# Find the best r (lowest RMSE) - if multiple options, select the smallest r
best_rmse = min(rmse_scores)
best_r_candidates = [r_values[i] for i, rmse in enumerate(rmse_scores) if rmse == best_rmse]
best_r = min(best_r_candidates)  # Select smallest r if multiple options

print(f"\\n🏆 BEST MODEL:")
print(f"r (alpha): {best_r}")
print(f"RMSE (Validation): {best_rmse}")

# Show all scores for comparison
print(f"\\n📊 COMPARISON OF ALL MODELS:")
print("-" * 40)
for i, (r, rmse) in enumerate(zip(r_values, rmse_scores)):
    marker = " ← BEST" if r == best_r else ""
    print(f"r = {r:>5}: RMSE = {rmse}{marker}")

print(f"\\n✅ ANSWER: r = {best_r} gives the best RMSE of {best_rmse}")
print(f"(If multiple r values have the same RMSE, we select the smallest r)")

Testing different r values for regularized linear regression:
Model                | r (alpha)  | RMSE (Validation)
Linear Regression    | 0          | 0.53           
Ridge Regression     | 0.01       | 0.53           
Ridge Regression     | 0.1        | 0.53           
Ridge Regression     | 1          | 0.53           
Ridge Regression     | 5          | 0.53           
Ridge Regression     | 10         | 0.53           
Ridge Regression     | 100        | 0.53           
\n🏆 BEST MODEL:
r (alpha): 0
RMSE (Validation): 0.53
\n📊 COMPARISON OF ALL MODELS:
----------------------------------------
r =     0: RMSE = 0.53 ← BEST
r =  0.01: RMSE = 0.53
r =   0.1: RMSE = 0.53
r =     1: RMSE = 0.53
r =     5: RMSE = 0.53
r =    10: RMSE = 0.53
r =   100: RMSE = 0.53
\n✅ ANSWER: r = 0 gives the best RMSE of 0.53
(If multiple r values have the same RMSE, we select the smallest r)


## Question 5: Model Stability

Assess model stability across different random seeds using 60%/20%/20% train/validation/test split."

In [10]:
# Question 5: Model Stability across different random seeds
# Use different seed values with 60%/20%/20% train/validation/test split

print("=== Question 5: Model Stability Analysis ===")

# Step 1: Prepare dataset - Fill missing values with 0
df_q5 = df.copy()
df_q5['horsepower'] = df_q5['horsepower'].fillna(0)

print(f"Dataset preparation:")
print(f"Shape: {df_q5.shape}")
print(f"Missing values after filling with 0: {df_q5.isnull().sum().sum()}")

# Step 2: Prepare features and target
X_q5 = df_q5.drop('fuel_efficiency_mpg', axis=1)
y_q5 = df_q5['fuel_efficiency_mpg']

print(f"\\nFeatures shape: {X_q5.shape}")
print(f"Target shape: {y_q5.shape}")
print(f"Feature columns: {list(X_q5.columns)}")

# Step 3: Test different seed values with train/validation/test split
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

print(f"\\nTesting with different random seeds:")
print("=" * 60)
print(f"{'Seed':<6} | {'Train Size':<10} | {'Val Size':<8} | {'Test Size':<9} | {'RMSE (Val)':<12}")
print("=" * 60)

for seed in seeds:
    # First split: separate test set (20%)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_q5, y_q5, test_size=0.2, random_state=seed
    )
    
    # Second split: from remaining 80%, split into train (60% of total) and validation (20% of total)
    # 60%/80% = 0.75 for train from the remaining data
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=seed  # 0.25 * 0.8 = 0.2 (20% of total)
    )
    
    # Train Linear Regression model (no regularization)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluate on VALIDATION dataset
    y_pred_val = model.predict(X_val)
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_scores.append(rmse_val)
    
    print(f"{seed:<6} | {len(X_train):<10} | {len(X_val):<8} | {len(X_test):<9} | {rmse_val:<12.6f}")

print("=" * 60)

# Step 4: Calculate standard deviation
std_rmse = np.std(rmse_scores)
std_rounded = round(std_rmse, 3)

print(f"\\n📊 RMSE SCORES ANALYSIS:")
print(f"All RMSE scores: {[round(score, 6) for score in rmse_scores]}")
print(f"Mean RMSE: {np.mean(rmse_scores):.6f}")
print(f"Standard deviation (np.std): {std_rmse:.6f}")
print(f"Standard deviation (rounded to 3 digits): {std_rounded}")

print(f"\\n✅ ANSWER: Standard deviation = {std_rounded}")

=== Question 5: Model Stability Analysis ===
Dataset preparation:
Shape: (9704, 5)
Missing values after filling with 0: 0
\nFeatures shape: (9704, 4)
Target shape: (9704,)
Feature columns: ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
\nTesting with different random seeds:
Seed   | Train Size | Val Size | Test Size | RMSE (Val)  
0      | 5822       | 1941     | 1941      | 0.528392    
1      | 5822       | 1941     | 1941      | 0.522328    
2      | 5822       | 1941     | 1941      | 0.515816    
3      | 5822       | 1941     | 1941      | 0.511901    
4      | 5822       | 1941     | 1941      | 0.519977    
5      | 5822       | 1941     | 1941      | 0.525629    
6      | 5822       | 1941     | 1941      | 0.507135    
7      | 5822       | 1941     | 1941      | 0.522171    
8      | 5822       | 1941     | 1941      | 0.515642    
9      | 5822       | 1941     | 1941      | 0.507455    
\n📊 RMSE SCORES ANALYSIS:
All RMSE scores: [0.528392, 0.522328, 

In [11]:
# Question 6: Final model with combined train+validation data
# Following exact instructions: use seed 9, combine train+validation, fill with 0, use r=0.001

print("=== Question 6: Final Model with Combined Training Data ===")

# Step 1: Prepare dataset - Fill missing values with 0
df_q6 = df.copy()
df_q6['horsepower'] = df_q6['horsepower'].fillna(0)

print(f"Dataset preparation:")
print(f"Shape: {df_q6.shape}")
print(f"Missing values after filling with 0: {df_q6['horsepower'].isnull().sum()}")

# Step 2: Prepare features and target
X_q6 = df_q6.drop('fuel_efficiency_mpg', axis=1)
y_q6 = df_q6['fuel_efficiency_mpg']

print(f"\nFeatures shape: {X_q6.shape}")
print(f"Target shape: {y_q6.shape}")

# Step 3: Split dataset like previously with seed 9 (60%/20%/20%)
# First split: separate test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_q6, y_q6, test_size=0.2, random_state=9
)

# Second split: from remaining 80%, split into train (60% of total) and validation (20% of total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=9  # 0.25 * 0.8 = 0.2 (20% of total)
)

print(f"\nData split with seed 9:")
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

# Step 4: Combine train and validation datasets
X_train_combined = pd.concat([X_train, X_val], axis=0)
y_train_combined = pd.concat([y_train, y_val], axis=0)

print(f"\nCombined training data:")
print(f"Features: {X_train_combined.shape}")
print(f"Target: {y_train_combined.shape}")

# Step 5: Train model with r=0.001 (Ridge Regression)
model_q6 = Ridge(alpha=0.001)
model_q6.fit(X_train_combined, y_train_combined)

print(f"\nModel training:")
print(f"Model: Ridge Regression with alpha=0.001")
print(f"Training set size: {len(X_train_combined)}")

# Step 6: Evaluate on test dataset
y_pred_test = model_q6.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\n📊 RESULTS:")
print(f"RMSE on test dataset: {rmse_test:.6f}")
print(f"RMSE rounded to 3 decimals: {rmse_test:.3f}")

print(f"\n✅ ANSWER: RMSE = {rmse_test:.3f}")

# Check which option it matches
options = [0.15, 0.515, 5.15, 51.5]
closest_option = min(options, key=lambda x: abs(x - rmse_test))
print(f"Closest option: {closest_option}")

=== Question 6: Final Model with Combined Training Data ===
Dataset preparation:
Shape: (9704, 5)
Missing values after filling with 0: 0

Features shape: (9704, 4)
Target shape: (9704,)

Data split with seed 9:
Training set: (5822, 4)
Validation set: (1941, 4)
Test set: (1941, 4)

Combined training data:
Features: (7763, 4)
Target: (7763,)

Model training:
Model: Ridge Regression with alpha=0.001
Training set size: 7763

📊 RESULTS:
RMSE on test dataset: 0.521034
RMSE rounded to 3 decimals: 0.521

✅ ANSWER: RMSE = 0.521
Closest option: 0.515
